In [2]:
from datasets import load_from_disk
dataset = load_from_disk('result/result')
import pandas as pd

df = pd.DataFrame()
df['full_text'] = [
    '\n\n'.join(
                [
                    "# Title\n{title}\n# Passage\n{passage}".format(
                        title=title,
                        passage=' '.join(sentences)
                    ) 
                    for title, sentences in zip(row["context"]["title"], row["context"]["sentences"])
                ]
                + [
                    '# Question\n{question}'.format(question=row["question"])
                ]
            )
    for row in dataset
]
df['text'] = [
    row["question"]
    for row in dataset
]
df['answer'] = [
    row['answer']
    for row in dataset
]
df['generated_answer'] = [
    row['generated_answer']
    for row in dataset
]
df['level'] = [
    row['level']
    for row in dataset
]
df['supporting_facts'] = [
    str(row['supporting_facts'])
    for row in dataset
]
df['id'] = [
    row['id']
    for row in dataset
]
df.to_csv("hotpotQA.csv", index=False)

/home/yiningho/workspace/datadisk/slicing/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from utils.compute_f1 import compute_f1
from utils.compute_f1 import normalize_text

In [3]:
precision = []
recall = []
f1_score = []
em = []
clean_answer = []
for i in range(len(dataset)):
    # print(dataset[i]['generated_answer'])
    # print('####')
    idx = dataset[i]['generated_answer'].find('# Answer')
    # print(dataset[i]['answer'])
    # print('####')
    if idx != -1:
        clean_answer.append(dataset[i]['generated_answer'][idx + 9:].split('\n')[0])
    else:
        clean_answer.append(dataset[i]['generated_answer'].split('\n')[0])
    if clean_answer[-1].lower().find("yes") == 0:
        clean_answer[-1] = "yes"
    elif clean_answer[-1].lower().find("no") == 0:
        clean_answer[-1] = "no"
    # print(clean_answer[i])
    # print("-------------------------")
    prec, rec, f1 = compute_f1(clean_answer[i], dataset[i]['answer'])
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)
    em.append(int(normalize_text(clean_answer[i]) == normalize_text(dataset[i]['answer'])))
    
    
# for i in range(300, 320):
#     row = dataset[i]
#     # print(row['generated_answer'])
#     # print('####')
#     print(row['answer'])
#     print('####')
#     print(row['generated_answer'])
#     print("-------------------------")

In [4]:
print(sum(precision) / len(precision))
print(sum(recall) / len(recall))
print(sum(f1_score) / len(f1_score))

print(sum(em) / len(em))

0.5867265567765568
0.6302671136653891
0.5841558527242717
0.44066666666666665


In [3]:
from utils.file import read_csv_file, read_txt_file

df = read_csv_file("result/result1.csv")
#random sample
# df = df.sample(n=500)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Columns: 157 entries, full_text to label_economy issues
dtypes: int64(75), object(82)
memory usage: 1.8+ MB
None


In [4]:
keywords = read_txt_file("data/valid_keywords.txt")
print(keywords)

['multiple languages', 'highly specific topics', 'non-specific topics', 'unspecific topics', 'multiple topics', 'everyday language', 'formal language', 'about movies', 'about books', 'about literature', 'about music', 'about rare topics', 'about common topics', 'with multiple intentions', 'with single intention', 'mention specific people', 'mention specific places', 'mention specific organizations', 'mention specific events', 'mention specific dates', 'mention specific numbers', 'mention specific quantities', 'explicitly biased', 'implicitly biased', 'explicitly subjective', 'implicitly subjective', 'explicitly objective', 'implicitly objective', 'positive sentiment', 'negative sentiment', 'neutral sentiment', 'with multiple sentiments', 'with single sentiment', 'widely known', 'not widely known', 'multiple choices', 'need domain-specific knowledge to understand', 'use ambiguous words', 'need logical reasoning to understand', 'contain homonyms', 'contain synonyms', 'contain antonyms', 

In [6]:
add_col = []
precision = []
recall = []
f1_score = []
em = []
clean = []
for index, row in df.iterrows():
    clean_answer = row['generated_answer']
    idx = clean_answer.find('# Answer')
    if idx != -1:
        clean_answer = clean_answer[idx + 9:].split('\n')[0]
    else:
        clean_answer = clean_answer.split('\n')[0]
    if clean_answer.lower().find("yes") == 0:
        clean_answer = "yes"
    elif clean_answer.lower().find("no") == 0:
        clean_answer = "no"
    prec, rec, f1 = compute_f1(clean_answer, row['answer'])
    add_col.append(clean_answer)
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)
    em.append(int(normalize_text(clean_answer) == normalize_text(row['answer'])))
    clean.append(clean_answer)
df['generated_answer'] = clean
    # print(row['label_{keyword}'.format(keyword='ambigious question_meta')])
print(sum(precision) / len(precision))
print(sum(recall) / len(recall))
print(sum(f1_score) / len(f1_score))
print(sum(em) / len(em))
df["clean_answer"] = add_col
df["precision"] = precision
df["recall"] = recall
df["f1"] = f1_score
df["em"] = em
# import random
# df['random'] = [random.randint(0, 1) for i in range(len(df))]
for keyword in keywords:
    true_df = df[df['label_{keyword}'.format(keyword=keyword)] == 1]
    false_df = df[df['label_{keyword}'.format(keyword=keyword)] != 1]
    print("keyword:", keyword)
    print("true slice:", len(true_df), "cases")
    print("prec", round(true_df["precision"].mean(), 4))
    print("rec", round(true_df["recall"].mean(), 4))
    print("f1", round(true_df["f1"].mean(), 4))
    print("em", round(true_df["em"].mean(), 4))
    print("false slice:", len(false_df), "cases")
    print("prec", round(false_df["precision"].mean(), 4))
    print("rec", round(false_df["recall"].mean(), 4))
    print("f1", round(false_df["f1"].mean(), 4))
    print("em", round(false_df["em"].mean(), 4))
    print("----")

0.5867265567765568
0.6302671136653891
0.5841558527242717
0.44066666666666665
keyword: multiple languages
true slice: 469 cases
prec 0.563
rec 0.5925
f1 0.5559
em 0.3923
false slice: 1031 cases
prec 0.5975
rec 0.6475
f1 0.597
em 0.4627
----
keyword: highly specific topics
true slice: 48 cases
prec 0.6128
rec 0.72
f1 0.6357
em 0.375
false slice: 1452 cases
prec 0.5859
rec 0.6273
f1 0.5825
em 0.4428
----
keyword: non-specific topics
true slice: 31 cases
prec 0.578
rec 0.6839
f1 0.6028
em 0.4516
false slice: 1469 cases
prec 0.5869
rec 0.6291
f1 0.5838
em 0.4404
----
keyword: unspecific topics
true slice: 49 cases
prec 0.2927
rec 0.3912
f1 0.3137
em 0.1224
false slice: 1451 cases
prec 0.5967
rec 0.6383
f1 0.5933
em 0.4514
----
keyword: multiple topics
true slice: 25 cases
prec 0.7133
rec 0.678
f1 0.6756
em 0.56
false slice: 1475 cases
prec 0.5846
rec 0.6295
f1 0.5826
em 0.4386
----
keyword: everyday language
true slice: 45 cases
prec 0.6315
rec 0.65
f1 0.6294
em 0.5111
false slice: 1455 cas

In [21]:
keywords = [
'non-specific topics',
'everyday language',
'formal language',
'about rare topics',
'mention specific quantities',
'explicitly biased',
'explicitly subjective',
'implicitly subjective',
'explicitly objective',
'implicitly objective',
'positive sentiment',
'negative sentiment',
'neutral sentiment',
'not widely known',
'need domain-specific knowledge to understand',
'contain homonyms',
'contain synonyms',
'contain antonyms',
'complex sentence structure',
'question start with "who"',
'question start with "what"',
'question start with "when"',
'question start with "where"',
'question start with "why"',
'question start with "how"',
'question start with "which"',
'question start with "is"',
'question start with "are"',
'question start with "do"',
'question start with "does"',
'question start with "has"',
'question start with "did"',
'question start with "have"',
'race issues',
'question start with "had"',
'question start with "can"',
'question start with "could"',
'question start with "should"',
'question start with "were"',
'question start with "would"',
'question start with "was"',
'question start with "will"',
]
df = df.drop(labels=['label_{key}_meta'.format(key=key) for key in keywords], axis=1)
df = df.drop(labels=['label_{key}'.format(key=key) for key in keywords], axis=1)

In [33]:
# print(df.columns.tolist())
# print(df.info())
# column = df.columns.tolist()
# for key in column:
#     if key.find("slice:") == 0:
#         df = df.rename(columns={key: 'slicing result for ' + key})
df.to_csv('final_result.csv')

In [6]:
print_keywords = [
    # "question that need commen sense to answer",
    # "multiple possible interpretations",
    'unspecific topics'
]

for keyword in print_keywords:
    true_df = df[df['label_{keyword}'.format(keyword=keyword)] == 1]
    false_df = df[df['label_{keyword}'.format(keyword=keyword)] != 1]
    print(keyword)
    random_true_df = true_df.sample(n=5)
    random_false_df = false_df.sample(n=5)
    print("True:")
    for index, row in random_true_df.iterrows():
        print("question:", row['text'])
        print("answer:", row['answer'])
        print("generated_answer:", row['clean_answer'])
        print("label reason:", row['label_{keyword}_meta'.format(keyword=keyword)])
        print("------------------------------------")
    print("\n\nFalse:")
    for index, row in random_false_df.iterrows():
        print("question:", row['text'])
        print("answer:", row['answer'])
        print("generated_answer:", row['clean_answer'])
        print("label reason:", row['label_{keyword}_meta'.format(keyword=keyword)])
        print("------------------------------------")

unspecific topics
True:
question: Grzegorz Piramowicz was a member of an Education group considered what?
answer: it is considered the first Ministry of Education in European history
generated_answer: Polish Enlightenment commission
label reason: My answer is yes. The text does not provide any specific details or context that would limit its applicability to a specific situation. The language used is general and could be applied to a wide range of situations where a group of people are considering something. For example, the text could be applied to a group of students discussing a school project, a group of colleagues brainstorming ideas for a work project, or a group of friends deciding on a vacation destination.
------------------------------------
question: How are Fred Guiol and Stuart Paton alike?
answer: film director and screenwriter.
generated_answer:  Both directed Laurel and Hardy films.
label reason: My answer is yes. The text provides general information about Fred Guiol a